In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.feature_selection import f_regression, mutual_info_regression
from itertools import combinations
from collections import Counter

DATA_DIR_NAME = '/Users/karenblakemore/Koverse/data/'

plt.rcParams.update({'figure.max_open_warning': 0})

## Data Preparation

In [ ]:
def data_preparation(pdf, drop_columns, numeric_columns):
    pdf = pdf.drop(drop_columns, axis=1)

    for col in pdf:
        if col in numeric_columns:
            print(col)
            pdf[col] = pd.to_numeric(pdf[col], downcast = 'integer')
        else:           
            pdf[col], _  = pd.factorize(pdf[col].values)

    return pdf

## Function to calculate correlations

In [ ]:
SCORE_COLUMNS = ['x', 'y', 'MI-Score', 'G-Test']

def calculate_scores(pdf, filter_nans):
    
    # Create pairs of all columns
    pairs = combinations(list(pdf.columns.values), 2)

    # Correlations dataframe
    scores_pdf = pd.DataFrame(columns=SCORE_COLUMNS)

    # Calculate MI & G-Test scores
    print('Calculating MI & G-Test Scores')
    for idx, pair in enumerate(pairs):
        pair_pdf = pdf[[pair[0], pair[1]]]  
        
         # Drop pairs with nan's
        if(filter_nans):
            pair_pdf = pair_pdf.dropna()
            
        N = pair_pdf.shape[0]
            
        var0_cardinality = pair_pdf[pair[0]].nunique(dropna=True)
        var1_cardinality = pair_pdf[pair[1]].nunique(dropna=True)
        
        mi_score = metrics.adjusted_mutual_info_score(pair_pdf[pair[0]], pair_pdf[pair[1]])
            
        frequency_df = pd.crosstab(pair_pdf[pair[0]], pair_pdf[pair[1]])

        frequency_df = frequency_df.loc[(frequency_df!=0).any(axis=1)]       # delete rows that are equal to zero
        frequency_df = frequency_df.loc[:, (frequency_df != 0).any(axis=0)]  # delete columns that are equal to zero

        frequency_matrix = pd.DataFrame.as_matrix(frequency_df)

        chi2, p, dof, expected = stats.chi2_contingency(frequency_matrix, lambda_='log-likelihood')

        crosstab_num_entries = var0_cardinality * var1_cardinality
        n = N/crosstab_num_entries    
        max_chi2 = ((N - n) ** 2)/n
        normalized_chi2 = chi2/max_chi2    
        
        score = {'x': pair[0], 
                 'y': pair[1], 
                 'MI-Score': mi_score,
                 'G-Test': normalized_chi2}

        scores_pdf = scores_pdf.append(score, ignore_index=True)
        
        # print every 10 calculations    
        if idx%10 == 0:   
            print(idx, mi_score, normalized_chi2, pair[0], pair[1])
    
    print(scores_pdf.shape)
    scores_pdf.describe()
    display(scores_pdf.head())
    
    return scores_pdf   

## Plot Bivariate Distributions

In [ ]:
MAX_NUNIQUE = 11

def bivariate_plots(pdf, scores_pdf, filter_nans):  
    for idx, row in scores_pdf.iterrows():
        x_var = row['x']
        y_var = row['y']
        
        print(x_var, y_var)
        pair_pdf = pdf[[x_var, y_var]]
         
        if filter_nans:
            pair_pdf = pair_pdf.dropna()
            
        x_nunique = pair_pdf[x_var].nunique()
        y_nunique = pair_pdf[y_var].nunique()

        if x_nunique > MAX_NUNIQUE:
            top_values = [x for x, count in Counter(pair_pdf[x_var]).most_common(MAX_NUNIQUE)]
            pair_pdf = pair_pdf[pair_pdf[x_var].isin(top_values)]
        if y_nunique > MAX_NUNIQUE:
            top_values = [y for y, count in Counter(pair_pdf[y_var]).most_common(MAX_NUNIQUE)]
            pair_pdf = pair_pdf[pair_pdf[y_var].isin(top_values)]
                
        cplot = sns.countplot(y=pair_pdf[x_var], hue=pair_pdf[y_var], data=pair_pdf)
        cplot.set_title('MI-Score={:.6f} G-Test={:.6f}'.format(row['MI-Score'], row['G-Test']))
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title=y_var)
        plt.show()      

## Bus Breakdowns and Delays
[Bys Breakdown and Delays](https://data.cityofnewyork.us/Transportation/Bus-Breakdown-and-Delays/ez4e-fazm)

### Load Data Set

In [ ]:
DATA_SET_NAME = 'bus_breakdown_and_delays'
pdf = pd.read_csv(DATA_DIR_NAME + DATA_SET_NAME + '.csv', encoding='latin-1')
print(pdf.shape)
display(pdf.head())
print(pdf.nunique())

In [ ]:
drop_columns = ['Busbreakdown_ID',
                'Created_On',
                'Has_Contractor_Notified_Parents',
                'Has_Contractor_Notified_Schools',
                'Have_You_Alerted_OPT',
                'How_Long_Delayed',
                'Incident_Number',
                'Informed_On',
                'Last_Updated_On',
                'Number_Of_Students_On_The_Bus',
                'Occurred_On',
                'Route_Number',
                'Schools_Serviced']
numeric_columns = []
              
prepped_pdf = data_preparation(pdf, drop_columns, numeric_columns)

display(prepped_pdf.head())

### Calculate Scores

In [ ]:
scores_pdf = calculate_scores(prepped_pdf, True)
display(scores_pdf.head())

In [ ]:
display(scores_pdf)

### Plot Results

In [ ]:
scores_pdf['max_score'] = scores_pdf[['MI-Score', 'G-Test']].max(axis=1)
bivariate_plots(pdf, scores_pdf.sort_values(by=['max_score'], ascending=False), True)

## Hospital Readmissions
The data set is a join of two on hospital name:
* [Hospital Readmission Reduction](https://healthdata.gov/dataset/hospital-readmission-reduction)
* [Hospital Ratings](https://www.kaggle.com/center-for-medicare-and-medicaid/hospital-ratings)

### Load Data Set

In [ ]:
DATA_SET_NAME = 'hospital_readmissions'
pdf = pd.read_csv(DATA_DIR_NAME + DATA_SET_NAME + '.csv', encoding='latin-1')
print(pdf.shape)
display(pdf.head())
print(pdf.nunique())

### Prepare Data Set

In [ ]:
pdf = pdf.replace({'Below the national average': 1,
                   'Same as the national average': 2, 
                   'Above the national average': 3,
                   'Not Available': np.nan,
                   'Too Few to Report': np.nan,
                   'Results are not available for this reporting period': np.nan
                  }
                 )

numeric_columns = ['Effectiveness of care national comparison',
                    'Efficient use of medical imaging national comparison',
                    'Hospital overall rating',
                    'Mortality national comparison',
                    'Patient experience national comparison',
                    'Readmission national comparison',
                    'Safety of care national comparison',
                    'Timeliness of care national comparison'
                   ]

drop_columns = ['Address',
                'City',
                'County Name',
                'Effectiveness of care national comparison footnote',
                'Efficient use of medical imaging national comparison footnote',
                'Emergency Services',
                'End Date',
                'Footnote',
                'Hospital Name',
                'Hospital overall rating footnote',
                'Measure Name',
                'Meets criteria for meaningful use of EHRs',
                'Mortality national comparison footnote',
                'Patient experience national comparison footnote',
                'Phone Number',
                'Provider ID',
                'Provider Number',
                'Readmission national comparison footnote',
                'Safety of care national comparison footnote',
                'Start Date',
                'State',
                'Timeliness of care national comparison footnote',
                'ZIP Code',
                'Hospital Name',
                'Hospital Type',
                'Excess Readmission Ratio',
                'Expected Readmission Rate',
                'Predicted Readmission Rate',
                'Number of Discharges',                   
                'Number of Readmissions']

prepped_pdf = data_preparation(pdf, drop_columns, numeric_columns)

display(prepped_pdf.head())

### Calculate Scores

In [ ]:
scores_pdf = calculate_scores(prepped_pdf, True)
display(scores_pdf.head())

### Plot results

In [ ]:
scores_pdf['max_score'] = scores_pdf[['MI-Score', 'G-Test']].max(axis=1)
bivariate_plots(pdf, scores_pdf.sort_values(by=['max_score'], ascending=False), True)